In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
from remove_chattering import remove_nuisance_alarms 
from openpyxl.styles import Font

In [ ]:
# This notebook creates a excel representation of alarm floods usefull for manual analysis

In [ ]:
alarms = pd.read_csv(f'../../data/CSD_similar_alarms_filtered.csv')

In [ ]:
device_floods = pd.read_csv(f'../../data/CSD_alarm_floods.csv')

In [ ]:
columnsToKeep = ["systemId", "deviceId", "alarmNumber", "level","description","startTimestamp", "endTimestamp"]
alarms = alarms[columnsToKeep]
alarms["startTimestamp"] = pd.to_datetime(alarms["startTimestamp"], errors='coerce')
alarms["endTimestamp"] = pd.to_datetime(alarms["endTimestamp"], errors='coerce')
alarms = alarms.dropna(subset=["startTimestamp", "endTimestamp"])
alarms = alarms.sort_values(by="startTimestamp")
    
device_floods["startTimestamp"] = pd.to_datetime(device_floods["startTimestamp"], errors='coerce')
device_floods["endTimestamp"] = pd.to_datetime(device_floods["endTimestamp"], errors='coerce')

In [ ]:
crane_floods = device_floods[device_floods["deviceId"].str.contains("Crane")].reset_index()
system_groupd = alarms.groupby("systemId")
useless_alarms = []
messages = ["Message"]
messages_to_keep = []

floods_to_label_df = pd.DataFrame()
for i, row in crane_floods.iterrows():
    systemId = row["deviceId"].split("_")[0]
    system_alarms = system_groupd.get_group(systemId)

    alarm_flood_alarms = system_alarms[(system_alarms["startTimestamp"] >= row["startTimestamp"]) & (system_alarms["startTimestamp"] <= row["endTimestamp"])]
    if len(alarm_flood_alarms) < 3:
        print(systemId, row["startTimestamp"], row["endTimestamp"])
    alarm_flood_alarms["flood_id"] = i
    floods_to_label_df = pd.concat((floods_to_label_df, alarm_flood_alarms))

floods_to_label_df["deviceId"] = floods_to_label_df["deviceId"] + "_" + floods_to_label_df["flood_id"].map(str)
floods_to_label_df =  remove_nuisance_alarms(floods_to_label_df, 10, messages, useless_alarms)
floods_to_label_df = floods_to_label_df.set_index("flood_id")
floods_to_label_df["startTimestamp"] = floods_to_label_df["startTimestamp"].dt.strftime("%Y-%m-%d %H:%M:%S.%f%z").str[:-8]
floods_to_label_df["endTimestamp"] = floods_to_label_df["endTimestamp"].dt.strftime("%Y-%m-%d %H:%M:%S.%f%z").str[:-8]
df = floods_to_label_df.reset_index()
df = df[["flood_id", "deviceId", "alarmNumber", "startTimestamp", "endTimestamp", "level", "description"]]

In [ ]:
# Optional initial clusters created with some clustering algorithm to help manual labeling
preclustering_labels = pd.read_csv("../../data/clusters/CSD_rule_labels.csv", index_col="flood_id", squeeze=True)

In [ ]:
# In this case some initial separation is done by hand and the data is read from the separate files
# Defined in the pre_clusters list
pre_clusters = []
pre_cluster_dict = {}
for cluster in pre_clusters:
    pre_cluster_dict[cluster] = pd.read_csv(f"", header=None)

In [ ]:
df["label"] = df["flood_id"].map(preclustering_labels)
def add_label(group):
    flood_id_to_label = { flood_id: pre_cluster_dict[group.name].iloc[i][0] for i, flood_id in enumerate(sorted(group["flood_id"].unique())) }
    group["cluster"] = group["flood_id"].map(flood_id_to_label)
    return group


df = df.groupby("label").apply(add_label)


In [ ]:
sorted_mc_clusters = df.sort_values(by=["label","cluster", "flood_id", "startTimestamp"])


In [ ]:
# Read additional process information usefull for manual labeling
statusesDict ={}
systems = [] # List of systemIds
for systemId in systems:
    for device in ["Crane", "System Device"]:
        deviceId = f"{systemId}_{device}"
        stat_df = pd.read_csv(f"../../data/CSD_statuses/statuses_{deviceId}.csv")
        stat_df["timestamp"] = pd.to_datetime(stat_df["timestamp"], errors='coerce')
        stat_df = stat_df.sort_values(by="timestamp")
        stat_df["timestamp"] = stat_df["timestamp"].dt.tz_localize(None)
        statusesDict[f"{systemId}_{device}"] = stat_df
        stat_df = stat_df.sort_values(by="timestamp")
        statusesDict[f"{systemId}_{device}"] = stat_df


In [ ]:
from collections import defaultdict

In [ ]:
# Add additional process info to help manual labeling
def attach_device_status(alarm_row):
    id = "_".join(alarm_row["deviceId"].split("_")[:-1])
    device_status_df = statusesDict[id]
    alarm_ts = pd.Timestamp(alarm_row["startTimestamp"])
    prev_statuses = device_status_df[(device_status_df["timestamp"] < alarm_ts) & (device_status_df["timestamp"] > alarm_ts - pd.Timedelta(seconds=300))]
    if len(prev_statuses) == 0:
        try:
            prev_statuses = device_status_df[(device_status_df["timestamp"] < alarm_ts)].iloc[-1]
            alarm_row["device_status"] = prev_statuses["status"]
            return alarm_row
        except:
            alarm_row["device_status"] = "No data"
            return alarm_row
    status_durations = defaultdict(int)
    status_durations[prev_statuses.iloc[0]["previousStatus"]] += (prev_statuses.iloc[0]["timestamp"]- alarm_ts + pd.Timedelta(seconds=300)).total_seconds()
    for i in range(len(prev_statuses)-1):
        status_durations[prev_statuses.iloc[i]["status"]] += (prev_statuses.iloc[i+1]["timestamp"] - prev_statuses.iloc[i]["timestamp"]).total_seconds()
    status_durations[prev_statuses.iloc[-1]["status"]] += (alarm_ts - prev_statuses.iloc[-1]["timestamp"]).total_seconds()

    most_common_status = max(status_durations, key=status_durations.get)
    alarm_row["device_status"] = most_common_status
    return alarm_row

In [ ]:
final_clusters = sorted_mc_clusters.apply(attach_device_status, axis=1)

In [ ]:
final_clusters[(final_clusters["flood_id"] > 239) & (final_clusters["flood_id"] < 248)]

In [ ]:
with pd.ExcelWriter(f"../../data/classification/clustered_CSD_alarm_floods.xlsx", engine='openpyxl') as writer:
    for name, g in final_clusters.groupby("label"):
        name = name.replace("Crane", "C").replace("System Device", "SD")[:30]
        new = pd.DataFrame()
        prev_flood_id = None
        for i, row in g.iterrows():
            if row["flood_id"] != prev_flood_id and prev_flood_id is not None:
                root_cause_row = pd.Series(["Root cause:"] + [pd.NA] * (len(sorted_mc_clusters.columns) - 1), index=sorted_mc_clusters.columns)
                new = new.append(root_cause_row, ignore_index=True)
                new = new.append(row, ignore_index=True)
            else:
                new = new.append(row, ignore_index=True)
            prev_flood_id = row["flood_id"]
        df = new[["cluster", "flood_id", "deviceId", "alarmNumber", "startTimestamp", "endTimestamp", "level", "device_status", "description"]]
        df.to_excel(writer, index=False, sheet_name=name)

        # Get the xlsxwriter workbook and worksheet objects
        workbook  = writer.book
        worksheet = writer.sheets[name]

        
        marker_font = Font(bold=True, color="FF0000")

        # Set column widths
        worksheet.column_dimensions['A'].width = 9
        worksheet.column_dimensions['B'].width = 15
        worksheet.column_dimensions['C'].width = 14
        worksheet.column_dimensions['D'].width = 13
        worksheet.column_dimensions['E'].width = 21
        worksheet.column_dimensions['F'].width = 21
        worksheet.column_dimensions['G'].width = 12
        worksheet.column_dimensions['H'].width = 12
        worksheet.column_dimensions['I'].width = 12
        worksheet.column_dimensions['J'].width = 12

            # Format root cause rows
        for row in range(df.shape[0]):
            if df.iloc[row]['flood_id'] == 'Root cause:':
                for col in range(1, df.shape[1]+1):  # 1-indexed for openpyxl
                    worksheet.cell(row=row+2, column=col).font = marker_font  # +2 to account for 1-indexing and header row
                    worksheet.row_dimensions[row+2].height = 50

